In [1]:
# this is some self-defined functions for testing the model

# These package is inherited from Lenard-Jones optimization part of DMFF

import openmm.app as app
import openmm as mm
import openmm.unit as unit
import numpy as np
import jax
import jax.numpy as jnp
import dmff
from dmff.api.xmlio import XMLIO
from dmff.api.paramset import ParamSet
from dmff.generators.classical import CoulombGenerator, LennardJonesGenerator
from dmff.api.hamiltonian import Hamiltonian
from dmff.operators import ParmedLennardJonesOperator
from dmff import NeighborListFreud
from dmff.mbar import ReweightEstimator
import mdtraj as md
from tqdm import tqdm, trange
import parmed
import sys
import os
from dmff.api.topology import DMFFTopology
# this is a package I write to solve some IO problems utils.py
from utils import create_supercell, gas_generate,add_loading, simple_merge
from utils import cutoff_topology
import matplotlib.pyplot as plt
import optax
from utils import extract_from_raspa
from IPython.display import display
from utils import scaling_gas, extract_from_raspa, write_scaling_gas

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.


In [7]:
structure_folder = "/home/yutao/project/Al-MOF/MIL-160/"
experiment_path = os.path.join(structure_folder, "298K_short.csv")
cif_path = os.path.join(structure_folder, "MIL-160-Al.cif")
dest_path = "/home/yutao/project/MIL-120/traj2/"
copy_to_path = "./traj2/"
ff_path = '/home/yutao/project/aiida/applications/ff_3.json'
"""

Superparameters for Lenard-Jone Potential optimization, some parameters need to read aiida workflow and set them

"""
Number_points = 4           ## must be smaller than len(picked_ls)
Trajectory_length = 250#250          #液体pdb文件的个数
loop_time =   100                  #迭代循环次数    推荐50-100


scaling_factors = (1,1,2)          # This is read from aiida workflow 2,2,2 for NOTT-300
cutoff = 0.905     #This value need to check. Because Openmm a little weired to compute the cutoff, for aiida, the cutoff is 12.0

Transfer_unit = 8.0284625000/9.6917060506 #It also depends on different structure
SET_temperature = 273
Framework_path = os.path.join(structure_folder,"structure.pdb")
Forcefiled_path = os.path.join(structure_folder,"forcefield.xml")
#In the whole workflow, new files will be written to the dest_path, and the original files will be copied to the copy_to_path
aiida_path = "/home/yutao/project/aiida/applications/"
Scaled_frame_path = os.path.join(structure_folder,"scaled_frame.pdb")

for direct in [dest_path, copy_to_path]:
    if not os.path.exists(direct):
        os.makedirs(direct)
        print("Create directory: ", direct)
'''

The format of experimental data: two columns which can be read by np.loadtxt without skiprows

'''

data = np.loadtxt(experiment_path, delimiter=',')
picked_ls = list(range(Number_points))#[0,1,2,3,4,5,6,7,8,9] #[0, 2, 4, 6, 8, 10, 14, 18, 22]#[0, 3, 6, 9, 12, 15, 18]
picked_pressure = [data[i,0] for i in picked_ls]
picked_isotherm = [data[i,1]*Transfer_unit for i in picked_ls]

In [8]:
from utils import write_force_field, write_pdb_file, rename_atoms, read_cif_file, transform_cif_info
from ase.io import read
from openmm import app
# co2 form TraPPE File, O17, C18 are just inherited from the first example: MIL-120 
co2_info = [{"name": "O17", "type": "O_co2", "charge": "-0.35"},
            {"name": "C18", "type": "C_co2", "charge": "0.70"}]

atoms = read(cif_path)
atomic_number = len(atoms)*scaling_factors[0]*scaling_factors[1]*scaling_factors[2]

cell_parameters = atoms.get_cell_lengths_and_angles() # Get the cell parameters
carterisian_pos = atoms.get_positions()
cif_info = read_cif_file(cif_path)
transformed_info = transform_cif_info(cif_info)
pos_info = rename_atoms(cif_info, carterisian_pos)

write_force_field(transformed_info, co2_info,Forcefiled_path)
write_pdb_file(pos_info,cell_parameters, Framework_path)


In [ ]:
"""

Superparameters for Lenard-Jone Potential optimization, some parameters need to read aiida workflow and set them

"""
Number_points = 1           ## must be smaller than len(picked_ls)
Trajectory_length = 250#250          #液体pdb文件的个数
loop_time =   100                  #迭代循环次数    推荐50-100
scaling_factors = (2,2,2)          # This is read from aiida workflow 2,2,2 for NOTT-300
cutoff = 0.905     #This value need to check. Because Openmm a little weired to compute the cutoff, for aiida, the cutoff is 12.0
experiment_path = "/home/yutao/project/Al-MOF/nott300/Default_Dataset.csv"
Transfer_unit = 8.0284625000/9.6917060506 #It also depends on different structure
SET_temperature = 273
Framework_path = "/home/yutao/project/Al-MOF/nott300/structure.pdb"
Forcefiled_path = "/home/yutao/project/Al-MOF/nott300/forcefield.xml"
#In the whole workflow, new files will be written to the dest_path, and the original files will be copied to the copy_to_path
aiida_path = "/home/yutao/project/aiida/applications/"
Scaled_frame_path = "/home/yutao/project/Al-MOF/nott300/scaled_frame.pdb"
dest_path = "/home/yutao/project/MIL-120/traj2/"
copy_to_path = "./traj2/"
ff_path = '/home/yutao/project/aiida/applications/ff_3.json'

for direct in [dest_path, copy_to_path]:
    if not os.path.exists(direct):
        os.makedirs(direct)
        print("Create directory: ", direct)